In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum

yu.setpath('general')

projs=['P0', 'Px', 'Py', 'Pz']
inserts=['tt', 'tx', 'ty', 'tz', 'xx', 'xy', 'xz', 'yy', 'yz', 'zz']
enss=['b','c','d','e']

# Conjugation

In [2]:
import random

m=random.random()
p1=[random.random(),random.random(),random.random()]; p2=[random.random(),random.random(),random.random()]

sgns=[]
for insert in inserts:
    proj='P0'
    t1=yum.ME2FF(m,p1,p2,proj,insert)
    t2=np.conj(yum.ME2FF(m,p2,p1,proj,insert))
    t=t1/t2
    sgn=1 if np.real(t[0])>0 else -1
    
    for proj in projs:
        t1=yum.ME2FF(m,p1,p2,proj,insert)
        t2=np.conj(yum.ME2FF(m,p2,p1,proj,insert))
        t=t1/t2
        assert(np.sum(np.abs(t-[sgn]*3))<1e-10)
        
    sgns.append(sgn)

display(pd.DataFrame([sgns],index=[''],columns=inserts))

,tt,tx,ty,tz,xx,xy,xz,yy,yz,zz
,1,-1,-1,-1,1,1,1,1,1,1


# Q2 distribution

In [3]:
# Q2 vs msq when sink=0
fig, axs = yu.getFigAxs(1,1,Lrow=4,Lcol=12)
ax=axs[0,0]

max_mom2_pc=81
range_xyz=range(-int(np.sqrt(max_mom2_pc))-1,int(np.sqrt(max_mom2_pc))+2)
moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

moms=[pf+pc for pf in [[0,0,0]] for pc in moms_pc]
moms=[list(ele) for ele in set([tuple(yu.mom3pt2standard(mom)) for mom in moms])]; moms.sort()
msqs=[mom[3]**2+mom[4]**2+mom[5]**2 for mom in moms]

msq_0p5=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=0.5]) for ens in enss]
msq_1=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=1]) for ens in enss]
msq_1p2=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=1.2]) for ens in enss]
msq_1p5=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=1.5]) for ens in enss]
msq_2=[np.max([msqs[i] for i in range(len(moms)) if yum.mom2Q2(moms[i],ens)<=2]) for ens in enss]

for iens,ens in enumerate(enss):
    Q2s=[yum.mom2Q2(mom,ens) for mom in moms]
    plt_x=msqs; plt_y=Q2s
    ax.plot(plt_x,plt_y,color=yu.colors8[iens],label=f'{ens}: L={yu.ens2NL[ens]*yu.ens2a[ens]:.3} fm; [{msq_0p5[iens]},{msq_1[iens]},{msq_1p2[iens]},{msq_1p5[iens]},{msq_2[iens]}]',ms=4)


yu.addRefLine(ax,0.5)
yu.addRefLine(ax,1)
yu.addRefLine(ax,1.2)
yu.addRefLine(ax,1.5)
yu.addRefLine(ax,2)
ax.legend()
ax.set_xlabel('msq')
ax.set_ylabel(r'$Q^2$ [GeV$^2$]')
fig.suptitle('Integers in legend indicate msq cuts for Q2 cuts at 0.5, 1, 1.2, 1.5, 2')
yu.finalizePlot('Q2_msq')

In [4]:
ens='b'
def getRank(moms,case):
    if case=='unequal':
        mpirs=[(mom,proj,insert,ri) for mom in moms for proj in projs for insert in inserts for ri in [0,1] if insert[0]!=insert[1] and yum.useQ(mom,proj,insert)[ri]]
        # print(mpirs)
    elif case=='equal':
        mpirs=[(mom,proj,insert,ri) for mom in moms for proj in projs for insert in inserts for ri in [0,1] if insert[0]==insert[1] and yum.useQ(mom,proj,insert)[ri]]
    elif case=='all':
        mpirs=[(mom,proj,insert,ri) for mom in moms for proj in projs for insert in inserts for ri in [0,1] if yum.useQ(mom,proj,insert)[ri]]
    G=np.array([yum.funcs_ri[ri](yum.ME2FF(939/yu.ens2aInv[ens],np.array(np.array(mom[:3])+mom[3:])*(2*np.pi/yu.ens2NL[ens]),(np.array(mom[:3]))*(2*np.pi/yu.ens2NL[ens]),proj,insert)) for mom,proj,insert,ri in mpirs])
    if len(G)!=0:
        U, S, VT = np.linalg.svd(G)
        tol = 1e-10
        rank = np.sum(S > tol)
        return rank
    return 0
def getRanks(moms):
    return [getRank(moms,case) for case in ['unequal','equal','all']]

res=yu.load_pkl_reg('n2qpp1s,2moms,2ranks')
res=None
if res is not None:
    n2qpp1s,n2qpp1_to_moms,n2qpp1_to_ranks=res
else:
    max_mom2_pc=27
    range_xyz=range(0,int(np.sqrt(max_mom2_pc))+2)
    moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

    max_mom2_pf=10
    range_xyz=range(-int(np.sqrt(max_mom2_pf))-1,int(np.sqrt(max_mom2_pf))+2)
    moms_pf=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pf]

    moms=[pf+pc for pf in moms_pf for pc in moms_pc]
    moms=[yu.mom3pt2standard_sym(mom) for mom in moms]
    moms=yu.removeDuplicates_moms(moms)
    moms=sorted(moms,key=yu.getSortkey_mom3pt)
    
    n2qpp1s_moms=[yu.mom2n2qpp1_sym(mom) for mom in moms]
    n2qpp1s=sorted(yu.removeDuplicates(n2qpp1s_moms), key=lambda x: (yum.n2qpp12Q2(x,ens),x[1]))

    n2qpp1_to_moms={n2qpp1:[] for n2qpp1 in n2qpp1s}
    for mom in moms:
        n2qpp1_to_moms[yu.mom2n2qpp1_sym(mom)].append(mom)

    n2qpp1_to_ranks={}
    for i,n2qpp1 in enumerate(n2qpp1s):
        print(f'{i}/{len(n2qpp1s)}',end='          \r')
        moms=n2qpp1_to_moms[n2qpp1]
        n2qpp1_to_ranks[n2qpp1]=getRanks(moms)
        
    yu.save_pkl_reg('n2qpp1s,2moms,2ranks',[n2qpp1s,n2qpp1_to_moms,n2qpp1_to_ranks])
    
    txts=[f'(n2q,n2p,n2p1) [Q2*1000 for {",".join(enss)}]: rank for [unequal,equal,all]; moms']
    for n2qpp1 in n2qpp1s:
        Q2s=','.join([f'{int(yum.n2qpp12Q2(n2qpp1,ens)*1000): 4d}' for ens in enss])
        txt=f'{n2qpp1} [{Q2s}]: {n2qpp1_to_ranks[n2qpp1]}; {n2qpp1_to_moms[n2qpp1]}'
        txts.append(txt)
    yu.save_txt_reg('n2qpp1s,2moms,2ranks',txts)

ranks_all=yu.removeDuplicates([tuple(n2qpp1_to_ranks[n2qpp1]) for n2qpp1 in n2qpp1s])
print(ranks_all)

[(2, 2, 3), (3, 2, 3), (2, 3, 3), (1, 2, 3), (2, 1, 3), (3, 3, 3), (1, 1, 1), (0, 1, 1)]


In [7]:
def ranks2fillstyle(ranks):
    r1,r2,r3=ranks
    t=(r1==3,r2==3,r3==3)
    
    fillstyle={
        (True,True,True):'full', # rank3 for both unequal and equal (therefore also all)
        (True,False,True):'left', # rank3 for unequal
        (False,True,True):'right', # rank3 for equal
        (False,False,True):'bottom', # rank3 for all
        (False,False,False):'none', # rank3 for no cases
    }[t]
    return fillstyle
fillstyles=['full','left','right','bottom','none']

fig, axs = yu.getFigAxs(1,1,Lrow=20,Lcol=30)
ax=axs[0,0]
ax.set_xlabel(r'$Q^2$ [GeV$^2$]')
ax.set_ylabel(r'Ensemble')
ax.set_ylim([-0.1,3.6])
ax.set_yticks([0,1,2,3])
ax.set_yticklabels(enss)

ax.set_xlim([-0.02,1.02])

p12s=range(11)

for iens,ens in enumerate(enss):
    for p12 in p12s:
        n2qpp1s_p12=[n2qpp1 for n2qpp1 in n2qpp1s if n2qpp1[-1]==p12]
        for fillstyle in fillstyles:
            n2qpp1s_p12_fillstyle=[n2qpp1 for n2qpp1 in n2qpp1s_p12 if ranks2fillstyle(n2qpp1_to_ranks[n2qpp1])==fillstyle]
            Q2s=[yum.n2qpp12Q2(n2qpp1,ens) for n2qpp1 in n2qpp1s_p12_fillstyle]
            
            xs=Q2s; ys=[iens+p12*0.05]*len(Q2s)
            ax.plot(xs,ys,color='b', marker=mpl.markers.MarkerStyle("s", fillstyle=fillstyle))
yu.finalizePlot('momDistribution')

In [ ]:
# def verify(mom):
#     momE=yu.mom_exchangeSourceSink(mom)
#     moms=sorted(yu.mom2moms(mom),key=yu.getSortkey_mom3pt)
#     momsE=sorted(yu.mom2moms(momE),key=yu.getSortkey_mom3pt)
#     momsEE=[yu.mom_exchangeSourceSink(mom) for mom in momsE]
#     assert(np.all(moms==momsEE) or np.all(moms==momsE))

# def verify(mom):
#     mom_std=yu.mom3pt2standard(mom)
#     momE=yu.mom_exchangeSourceSink(mom)
#     momE_std=yu.mom3pt2standard(momE)
#     momE_stdE=yu.mom_exchangeSourceSink(momE_std)
#     assert(np.all(mom_std==momE_stdE) or np.all(mom_std==momE_std))
   
# max_mom2_pc=27
# range_xyz=range(0,int(np.sqrt(max_mom2_pc))+2)
# moms_pc=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pc]

# max_mom2_pf=10
# range_xyz=range(-int(np.sqrt(max_mom2_pf))-1,int(np.sqrt(max_mom2_pf))+2)
# moms_pf=[[x,y,z] for x in range_xyz for y in range_xyz for z in range_xyz if x**2+y**2+z**2<=max_mom2_pf]

# moms=[pf+pc for pf in moms_pf for pc in moms_pc]

# for i,mom in enumerate(moms):
#     print(f'{i}/{len(moms)}',end='             \r')
#     verify(mom)